### [참고] <a href="https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">Pandas Cheat Sheet</a>

#### https://pandas.pydata.org/docs/user_guide/text.html

**str 메소드**
- pandas.core.strings.StringMethods 의 별칭
- 호출할 수 없기 때문에 괄호를 사용하지 않음

In [1]:
import pandas as pd
import numpy as np

In [2]:
ser = pd.Series(['Suho','AA',np.nan,'Rabbit'])
ser

0      Suho
1        AA
2       NaN
3    Rabbit
dtype: object

#### 1) lower() / upper() / len()

In [3]:
ser.str.lower()

0      suho
1        aa
2       NaN
3    rabbit
dtype: object

In [4]:
ser.str.upper()

0      SUHO
1        AA
2       NaN
3    RABBIT
dtype: object

In [5]:
ser.str.len()

0    4.0
1    2.0
2    NaN
3    6.0
dtype: float64

#### 2) strip()

In [6]:
ser = pd.Series(['Suho       ','AA',np.nan,'        Rabbit'])
ser

0       Suho       
1                AA
2               NaN
3            Rabbit
dtype: object

In [7]:
ser.str.strip()

0      Suho
1        AA
2       NaN
3    Rabbit
dtype: object

In [8]:
df = pd.DataFrame(np.random.randn(2,2), columns=['  Column A ',' Column B '])
df

,Column A,Column B
0,0.835923,0.335094
1,-1.219189,0.980892


In [9]:
df.columns

Index(['  Column A ', ' Column B '], dtype='object')

In [10]:
df['  Column A ']

0    0.835923
1   -1.219189
Name:   Column A , dtype: float64

In [11]:
df.columns.str.strip()

Index(['Column A', 'Column B'], dtype='object')

In [12]:
df.columns.str.lower()

Index(['  column a ', ' column b '], dtype='object')

In [13]:
# 컬렴명 양쪽 공백 제거, 전체 소문자, 컬럼명 사이의 공백은 _대체
df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')
df.columns

Index(['column_a', 'column_b'], dtype='object')

#### 3) split()

In [14]:
ser = pd.Series(['ha_a_b','hi_c_d',np.nan, ' ho_e_f'])
ser

0     ha_a_b
1     hi_c_d
2        NaN
3     ho_e_f
dtype: object

In [15]:
ser.str.split('_')

0     [ha, a, b]
1     [hi, c, d]
2            NaN
3    [ ho, e, f]
dtype: object

* **expand=True** 결과를 데이터 프레임으로 돌려줌

In [16]:
ser.str.split("_",expand=True)

,0,1,2
0,ha,a,b
1,hi,c,d
2,NaN,NaN,NaN
3,ho,e,f


#### 4) replace()

In [17]:
ser = pd.Series(['Suho','bAAa',np.nan,'cute_dog'])
ser

0        Suho
1        bAAa
2         NaN
3    cute_dog
dtype: object

In [18]:
ser.str.replace('^.a|dog','***',case=False,regex=True)

0        Suho
1       ***Aa
2         NaN
3    cute_***
dtype: object

- ^는 입력라인의 시작
- .a 는 맨 첫 문자를 포함하고 a로 끝나는 부분을 매칭
- | or 의 의미
- dog에 해당하는 부분 매칭
- case=False 대소문자 구분 안함
- regex=True 전달된 패턴이 정규식

#### 5) cat : 텍스트 이어 붙이기 

In [20]:
ser = pd.Series(['ha','hi','ho'])
ser

0    ha
1    hi
2    ho
dtype: object

In [22]:
ser.str.cat(sep=',')

'ha,hi,ho'

In [23]:
ser.str.cat()

'hahiho'

* **손실값이 있다면 연결 안함**

In [24]:
ser = pd.Series(['ha',np.nan,'ho'])
ser

0     ha
1    NaN
2     ho
dtype: object

In [25]:
ser.str.cat(sep=',')

'ha,ho'

In [26]:
ser.str.cat(sep=',',na_rep='*')

'ha,*,ho'

### [실습2] 일기 형식의 데이터 가공

In [34]:
day_plan = {
    '1st_seq : getting up at 05:45am',
    '2st_seq : swimming from 06:00 to 07:00',
    '3rd_seq : My morning food is American style',
    '4th_seq : Writing some proposal from 02:00pm to 06:00pm',
    '5th_seq : Arriving at JongGak at 07:00pm',
    '6th_seq : Fun with friends enjoy beer till 09:30pm',
    '7th_seq : My house at 10:30pm and sleeping by 12:00pm'
}
df = pd.DataFrame(day_plan, columns=['schedule'])
df

,schedule
0,6th_seq : Fun with friends enjoy beer till 09:...
1,7th_seq : My house at 10:30pm and sleeping by ...
2,1st_seq : getting up at 05:45am
3,2st_seq : swimming from 06:00 to 07:00
4,5th_seq : Arriving at JongGak at 07:00pm
5,4th_seq : Writing some proposal from 02:00pm t...
6,3rd_seq : My morning food is American style


In [35]:
# split() : 공백 기준 

df['schedule'].str.split()

0    [6th_seq, :, Fun, with, friends, enjoy, beer, ...
1    [7th_seq, :, My, house, at, 10:30pm, and, slee...
2               [1st_seq, :, getting, up, at, 05:45am]
3       [2st_seq, :, swimming, from, 06:00, to, 07:00]
4     [5th_seq, :, Arriving, at, JongGak, at, 07:00pm]
5    [4th_seq, :, Writing, some, proposal, from, 02...
6    [3rd_seq, :, My, morning, food, is, American, ...
Name: schedule, dtype: object

In [36]:
# 공백기준으로 분할된 문자열의 수 파악

df['schedule'].str.split().str.len()

0     9
1    10
2     6
3     7
4     7
5     9
6     8
Name: schedule, dtype: int64

#### contains() : 특정 문자열을 가지고 있는지 확인

In [37]:
df['schedule'].str.contains('My')

0    False
1     True
2    False
3    False
4    False
5    False
6     True
Name: schedule, dtype: bool

#### count()

In [39]:
# 숫자가 몇 개 있는지 확인

df['schedule'].str.count('\d') # [0-9]

0    5
1    9
2    5
3    9
4    5
5    9
6    1
Name: schedule, dtype: int64

#### findall()

In [41]:
df['schedule'].str.findall('(\d\d):(\d\d)')

0              [(09, 30)]
1    [(10, 30), (12, 00)]
2              [(05, 45)]
3    [(06, 00), (07, 00)]
4              [(07, 00)]
5    [(02, 00), (06, 00)]
6                      []
Name: schedule, dtype: object

In [ ]:
df['schedule'].str.findall('\d')